# Dense Vector

In [1]:
from elasticsearch import Elasticsearch
from pprint import pprint

In [2]:
es=Elasticsearch("http://localhost:9200")
client_info=es.info()
print("Connected to the elastic search client !")
pprint(client_info.body)

Connected to the elastic search client !
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'XoT9Xm1xR3O6L3zfFdu5nQ',
 'name': 'd8153502b8b9',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-02-13T09:35:20.314882762Z',
             'build_flavor': 'default',
             'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.4.2',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.6.2'}}


## Create an index for dense Vector storage
---

In [8]:
index_name="dense_vector_index"

# delete the index if already exists
es.indices.delete(index= index_name, ignore_unavailable= True)

# create the index

es.indices.create(index= index_name, mappings={
    "properties":{
        "sides_length":{
            "type":"dense_vector",
            "dims":4
        },
        "shape":{
            "type":"keyword"
        }
    }
})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'dense_vector_index'})

### 1. valid cases
-   insert a doc for square shape 

In [10]:
res=es.index(
    index=index_name,
    document={
        "shape":"square",
        "sides_length":[5,5,5,5]
        },
    id="2"
    )
res.body

{'_index': 'dense_vector_index',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

- invalid cases

In [11]:
res=es.index(
    index=index_name,
    id="3",
    document={
        "shape":"pentagon",
        "sides_length":[5,5,5,5,5]
    }
)

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'failed to parse')

In [12]:
res=es.index(
    index=index_name,
    id="3",
    document={
        "shape":"polygon",
        "sides_length":[[5,5],[5,5],5]
    }
)

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [START_ARRAY]')